In [1]:
import os
from dotenv import load_dotenv
import json
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain_core.callbacks import BaseCallbackHandler

from ragas import evaluate
from ragas.run_config import RunConfig
from ragas.metrics import faithfulness, answer_relevancy, answer_correctness, context_precision, context_recall, answer_similarity
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from datasets import Dataset
import itertools
import pandas as pd

In [2]:
load_dotenv()

True

In [3]:
local_path = "../pdf/BILLS-119hr1eh.pdf"

# Parameters to reduce runtime of the PDF loader
# split_pdf_page=True
# split_pdf_concurrency_level = 15

if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path,
                                   split_pdf_page=True,
                                   split_pdf_concurrency_level=15)
    data = loader.load()
else:
    print("Upload a PDF file for processing.")

In [4]:
len(data[0].page_content)

1136279

In [5]:
#Split and chunk the data
chunk_size = 750
chunk_overlap = 200

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_documents(data)

# Add the chunks to vector database, which uses nomic for model embeddings
vector_db = Chroma.from_documents(
                                    documents=chunks, 
                                    embedding=OllamaEmbeddings(model="nomic-embed-text"),
                                    collection_name="local-rag"
                                )

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [6]:
local_llm = "llama3.2"
llm = ChatOllama(model=local_llm)

# Set up a basic PromptTemplate as the backbones of the solution
# Ask the system to gather several responses and to limit response to 200 words
QUERY_PROMPT = PromptTemplate(
    input_variables = ["question"],
        template="""You are an AI Language model assistant. Your task is to generate three different versions of the given user question 
        to retrieve relevant documents from a vector database. Please be as concise as possible and limit your response to 200 words or less. 
        Original question: {question} """
)

retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(),llm, prompt=QUERY_PROMPT)

In [7]:
# use a ChatPromptTemplate to initiate a conversation, allowing the System to assume a Role
chat_template = """Answer the question based only on the following context: 
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(chat_template)

chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

### Context-Driven responses

In [ ]:
# q_list = ['Resulting from this Act, which groups would be harmed most?',\
#           'Resulting from this Act, which groups would benefit most?']

# c_list = ['Role: you are a staunch Democrat',\
#           'Role: you are a staunch Republican',\
#           'Role: you are a US citizen with no political affiliation',\
#           'Role: you are a wealthy investor with interests in oil, gas, and mining',\
#           'Role: you are a person that enjoys recreating in public lands while living in Colorado']

In [8]:
def response_gen(q, context):
    """
    Invoke the language model chain with a given question and context, 
    then print the question, context, and the model's response.

    Args:
        q (str): The question to ask.
        context (str): The context or role to provide to the model.
    """
    response = chain.invoke(input={'context': context, 'question': q})
    
    print('*** \n')
    print(f"Question - {q}")
    print(f"Context - {context} \n")
    print("Response: \n", response)

    return response


def iterate_responses(q_list, c_list):
    """
    Iterate over all combinations of questions and contexts, generate responses using the language model,
    and return a DataFrame with the results.

    Args:
        q_list (list): List of questions to ask.
        c_list (list): List of contexts or roles to provide to the model.

    Returns:
        pd.DataFrame: DataFrame containing columns 'question', 'context', and 'response' for each combination.
    """
    response_list = []
    df_responses = []
    
    for combo in itertools.product(q_list, c_list):
        response = response_gen(q=combo[0], context=combo[1])

        data = {
        'question': [combo[0]],
        'context': [combo[1]],
        'response': [response]
        }
    
        df = pd.DataFrame(data)
    
        response_list.append(df)

    df_responses = pd.concat(response_list)
    df_responses.reset_index(inplace=True, drop=True)

    return df_responses

In [9]:
q_list1 = ['Resulting from this Act, which groups would be harmed most?']

c_comp_list = ['Role: you are a US citizen with no political affiliation',\
        'Role: you are a staunch Republican',\
        'Role: you are a staunch Republican',\
        'Role: you are a staunch Democrat',\
        'Role: you are a staunch Republican and Trump Supporter']

df_responses = iterate_responses(q_list=q_list1, c_list=c_comp_list)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


*** 

Question - Resulting from this Act, which groups would be harmed most?
Context - Role: you are a US citizen with no political affiliation 

Response: 
 Based on the text of the Act, it appears that the following groups would be harmed most:

1. Immigrants and asylum seekers who may be impacted by changes to parole policies and increased scrutiny.
2. Low-income individuals who may struggle with rising healthcare costs due to reduced availability of affordable prescription drugs.
3. Small businesses and independent pharmacies who may face increased regulatory burdens and costs under new laws related to exchange enrollment verification, exchange vendor reporting, and extra-long staple cotton pricing.
4. Farmers and producers who may be affected by changes to agricultural subsidies, land use regulations, and environmental policies.

It is worth noting that these are just potential examples of groups that could be harmed by certain provisions in the Act, and it's impossible to know fo

In [10]:
pd.set_option('display.max_colwidth', None)
df_responses.head()

,question,context,response
0,"Resulting from this Act, which groups would be harmed most?",Role: you are a US citizen with no political affiliation,"Based on the text of the Act, it appears that the following groups would be harmed most:\n\n1. Immigrants and asylum seekers who may be impacted by changes to parole policies and increased scrutiny.\n2. Low-income individuals who may struggle with rising healthcare costs due to reduced availability of affordable prescription drugs.\n3. Small businesses and independent pharmacies who may face increased regulatory burdens and costs under new laws related to exchange enrollment verification, exchange vendor reporting, and extra-long staple cotton pricing.\n4. Farmers and producers who may be affected by changes to agricultural subsidies, land use regulations, and environmental policies.\n\nIt is worth noting that these are just potential examples of groups that could be harmed by certain provisions in the Act, and it's impossible to know for certain without more information about the specific provisions and their intended effects."
1,"Resulting from this Act, which groups would be harmed most?",Role: you are a staunch Republican,"As a staunch Republican, I would argue that certain groups would be harmed most by the provisions of this Act. Specifically, I would point to the following groups:\n\n1. Small business owners and farmers: The Act's emphasis on increasing the minimum wage and implementing stricter regulations could lead to increased costs for small businesses and farmers, making it more difficult for them to compete with larger corporations.\n2. Low-income households: Some provisions of the Act, such as the expansion of Medicaid and the increase in the minimum wage, could lead to higher prices for goods and services, which could disproportionately affect low-income households who already struggle to make ends meet.\n3. Rural communities: The Act's focus on urban areas and its neglect of rural areas could lead to a lack of investment and opportunities in rural communities, exacerbating existing disparities between urban and rural areas.\n\nThese groups would likely be harmed most by the provisions of this Act, as they are often vulnerable to economic downturns and regulatory changes."
2,"Resulting from this Act, which groups would be harmed most?",Role: you are a staunch Republican,"As a staunch Republican, I would likely believe that certain groups, such as those who benefit from government assistance programs or regulations, would be harmed most by the resulting policies and provisions of this Act.\n\nBased on the text provided, some specific groups that may be harmed include:\n\n* Low-income individuals and families: The Act's provision to disallow a premium tax credit during periods of Medicaid ineligibility due to alien status could negatively impact low-income individuals who rely on government assistance programs.\n* Rural communities: The Act's designation of certain census tracts as ""qualified opportunity zones"" may have unintended consequences for rural communities, potentially exacerbating existing economic disparities.\n* Small businesses and entrepreneurs: The Act's provisions related to exchange enrollment verification and controlled groups may create additional regulatory burdens and complexities for small businesses and entrepreneurs.\n\nHowever, it is essential to note that these groups' interests may be influenced by one's individual political perspective. As a Republican, I would likely prioritize the interests of individuals who are not reliant on government assistance programs or regulations, such as those in rural areas with limited access to healthcare or economic opportunities."
3,"Resulting from this Act, which groups would be harmed most?",Role: you are a staunch Democrat,"Based on the provided context, I can infer that the Act in question is H.R. 1 EH1S (119th Congress), also known as the Inflation Reduction Act of 2022.\n\nConsidering my role as a

### Leverage RAGAS for Evaluating Prompts

In [11]:
def structure_eval_data(question, contexts, answer, ground_truth):
    """
    Structure the evaluation data into a Dataset format.
    
    Args:
        question (str): The evaluation question.
        contexts (list): List of contexts for the question.
        answer (str): The answer for the question.
        ground_truth (str): The ground truth for the evaluation.

    Returns:
        Dataset: A structured dataset containing the evaluation data.
    """
    eval_data = {
        'question': [question],
        'answer': [answer],
        'contexts': [contexts],
        'ground_truth': [ground_truth]
    }

    json_formatted_string = json.dumps(eval_data, indent=4)
    print("Structured Evaluation Data (in json format):")
    print(json_formatted_string)

    return Dataset.from_dict(eval_data)

In [12]:
class TestCallback(BaseCallbackHandler):

    def on_llm_start(self, serialized, prompts, **kwargs):
        print(f"**********Prompts*********:\n {prompts[0]}\n\n")

    def on_llm_end(self, response, **kwargs):
        print(f"**********Response**********:\n {response}\n\n")

In [13]:
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o", temperature=0.0, max_tokens=3000))
evaluator_embed = LangchainEmbeddingsWrapper(OllamaEmbeddings(model="llama3.2"))

In [14]:
def evaluate_llm_with_ragas(dataset, metrics, llm, embeddings, run_config, raise_exceptions=True, callbacks=None):
    """
    Evaluate a dataset using RAGAS metrics and print the results.

    Args:
        dataset: The dataset to evaluate.
        metrics: List of metrics to use for evaluation.
        llm: The language model wrapper for evaluation.
        embeddings: The embeddings wrapper for evaluation.
        run_config: The run configuration for evaluation.
        raise_exceptions (bool, optional): Whether to raise exceptions during evaluation. Defaults to True.
        callbacks (list, optional): List of callback handlers. Defaults to None.

    Returns:
        None. Prints the evaluation result.
    """
    # Evaluate using RAGAS
    result = evaluate(
        dataset=dataset,
        metrics=metrics,
        llm=llm,
        embeddings=embeddings,
        run_config=run_config,
        raise_exceptions=raise_exceptions,
        callbacks=callbacks
    )

    df = result.to_pandas()
    print(df.head())


In [15]:
dataset_repub = structure_eval_data(question="Resulting from this Act, which groups would be harmed most?", 
                              contexts=["Role: you are a staunch Republican"], 
                              answer="Based on the provided context, as a staunch Republican, I would argue that the groups most harmed by the resulting Act would likely be small business owners and rural communities. The Act appears to include several provisions aimed at expanding access to healthcare and addressing issues related to poverty and inequality. Specifically, Sections 111201 and 111202 of the Act expand the definition of 'rural emergency hospital' under the Medicare program, which could provide additional support for rural hospitals that serve underserved communities. Additionally, Section 44301 expands the exclusion for orphan drugs under the Drug Price Negotiation Program, which could help reduce costs for some patients who rely on these medications. However, it's also worth noting that some provisions of the Act, such as the elimination of certain tax credits and deductions, could have a negative impact on small business owners and low-income families. For example, Section 112005 terminates the energy efficient home improvement credit, which could make it more difficult for low-income households to access affordable housing options.\n\nOverall, while there may be some benefits to certain groups under the Act, I believe that small business owners and rural communities would be among the most harmed by the resulting changes.",
                              ground_truth="""Foreign nationals: The unfair foreign tax provision may affect foreign nationals who are subject to taxation by a foreign government.\
            Small businesses: Some provisions, such as the repeal of EPA rules and NHTSA standards, may benefit small businesses by reducing regulations and costs.\
            Low-income individuals: The legislation includes provisions related to health insurance, such as the exchange enrollment verification requirement and the premium adjustment percentage. These provisions may affect low-income individuals who rely on government-subsidized health insurance plans.\
            Environmental groups and advocates: The Act repeals EPA rules related to greenhouse gas emissions standards (Section 42201), which could harm environmental organizations that relied on these regulations to advocate for climate action.\
            Alien populations: The Act revises the definition of "eligible alien" in certain contexts, which could affect non-citizen populations who rely on specific healthcare programs or services.\
            Health care workers: The legislation includes provisions that may impact health care workers, particularly those involved in administering or managing government-subsidized health insurance programs.\
            Clean energy companies: The legislation includes provisions that may impact clean energy companies, particularly those involved in renewable energy projects and environmental regulations."""
                              )

Structured Evaluation Data (in json format):
{
    "question": [
        "Resulting from this Act, which groups would be harmed most?"
    ],
    "answer": [
        "Based on the provided context, as a staunch Republican, I would argue that the groups most harmed by the resulting Act would likely be small business owners and rural communities. The Act appears to include several provisions aimed at expanding access to healthcare and addressing issues related to poverty and inequality. Specifically, Sections 111201 and 111202 of the Act expand the definition of 'rural emergency hospital' under the Medicare program, which could provide additional support for rural hospitals that serve underserved communities. Additionally, Section 44301 expands the exclusion for orphan drugs under the Drug Price Negotiation Program, which could help reduce costs for some patients who rely on these medications. However, it's also worth noting that some provisions of the Act, such as the elimination of cer

In [16]:
dataset_democrat = structure_eval_data(question="Resulting from this Act, which groups would be harmed most?", 
                              contexts=["Role: you are a staunch Democrat"], 
                              answer="Based on my understanding of the content of the provided text, it appears that the Act is primarily focused on tax reform and regulatory changes.\
                                However, some provisions in the Act could potentially harm certain groups. Some potential groups that might be harmed by this Act include:\
                                    * Low- and middle-income individuals who would face increased taxes under certain provisions\
                                    * Environmentally conscious individuals who would see stricter regulations relaxed regarding greenhouse gas emissions standards for light-duty vehicles\
                                    * Workers in the healthcare industry who may face changes to their job security due to the new requirements on exchange enrollment verification\
                                    * Certain corporations or organizations that might be affected by the new rules on foreign research and experimental expenditures, as well as the restriction on donations made pursuant to settlement agreements.",
                              ground_truth="""Foreign nationals: The unfair foreign tax provision may affect foreign nationals who are subject to taxation by a foreign government.\
            Small businesses: Some provisions, such as the repeal of EPA rules and NHTSA standards, may benefit small businesses by reducing regulations and costs.\
            Low-income individuals: The legislation includes provisions related to health insurance, such as the exchange enrollment verification requirement and the premium adjustment percentage. These provisions may affect low-income individuals who rely on government-subsidized health insurance plans.\
            Environmental groups and advocates: The Act repeals EPA rules related to greenhouse gas emissions standards (Section 42201), which could harm environmental organizations that relied on these regulations to advocate for climate action.\
            Alien populations: The Act revises the definition of "eligible alien" in certain contexts, which could affect non-citizen populations who rely on specific healthcare programs or services.\
            Health care workers: The legislation includes provisions that may impact health care workers, particularly those involved in administering or managing government-subsidized health insurance programs.\
            Clean energy companies: The legislation includes provisions that may impact clean energy companies, particularly those involved in renewable energy projects and environmental regulations."""
                              )

Structured Evaluation Data (in json format):
{
    "question": [
        "Resulting from this Act, which groups would be harmed most?"
    ],
    "answer": [
        "Based on my understanding of the content of the provided text, it appears that the Act is primarily focused on tax reform and regulatory changes.                                However, some provisions in the Act could potentially harm certain groups. Some potential groups that might be harmed by this Act include:                                    * Low- and middle-income individuals who would face increased taxes under certain provisions                                    * Environmentally conscious individuals who would see stricter regulations relaxed regarding greenhouse gas emissions standards for light-duty vehicles                                    * Workers in the healthcare industry who may face changes to their job security due to the new requirements on exchange enrollment verification                        

In [17]:
evaluate_llm_with_ragas(
    dataset=dataset_repub, 
    metrics=[
        #faithfulness, 
        answer_relevancy, 
        answer_correctness, 
        #context_precision, 
        #context_recall, 
        answer_similarity
    ], 
    llm=evaluator_llm, 
    embeddings=evaluator_embed, 
    run_config=RunConfig(timeout=300, max_retries=10, max_wait=300, log_tenacity=False), 
    raise_exceptions=True, 
    callbacks=None
)

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

                                                    user_input  \
0  Resulting from this Act, which groups would be harmed most?   

                     retrieved_contexts  \
0  [Role: you are a staunch Republican]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [18]:
evaluate_llm_with_ragas(
    dataset=dataset_democrat, 
    metrics=[
        #faithfulness, 
        answer_relevancy, 
        answer_correctness, 
        #context_precision, 
        #context_recall, 
        answer_similarity
    ], 
    llm=evaluator_llm, 
    embeddings=evaluator_embed, 
    run_config=RunConfig(timeout=300, max_retries=10, max_wait=300, log_tenacity=False), 
    raise_exceptions=True, 
    callbacks=None
)

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

                                                    user_input  \
0  Resulting from this Act, which groups would be harmed most?   

                   retrieved_contexts  \
0  [Role: you are a staunch Democrat]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                